In [1]:
# TODO come up wiht good way to compare models!!!!


# TODO Get rid of scaling. 
# TODO talk to marco about consumption.
# TODO make average model the baseline.
# TODO make scoring the parent class. Give only fitting and _func as input. 



# TODO, build some resistance against unseen actions into the thermal model
import sys
sys.path.append("..")
sys.path.append("../MPC")

from ControllerDataManager import ControllerDataManager
from ThermalModel import *
import pandas as pd 
import numpy as np
import pickle
import datetime
import yaml
import pytz
import pprint
import datetime



import matplotlib.pyplot as plt
from xbos import get_client

from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle


In [2]:
from TempThermalModel import ThermalModel
from AverageThermalModel import ThermalModel as AverageThermalModel
from AverageThermalModel import AverageMPCThermalModel as AverageMPCThermalModel
# from ActionThermalModel import ThermalModel

In [3]:
# TODO instead of hardcoding input. just give it a dictionary it should append. 
def save_thermal_model_data(thermalModel, corresponding_days, params, file_name):
    """saves the whole model to a yaml file.
    RECOMMENDED: PYAML should be installed for prettier config file.
    param: thermalModel : object, thermal model which stores the types of scoring.
    param: corresponding_days: list, the order of data according to the number of days from current day.
    param: params, the past parameters"""
    config_dict = {}
    
    config_dict["Param Order"] = thermalModel._params_order
    config_dict["Params"] = params
    

    # store evaluations and RMSE's.
    config_dict["Evaluations"] = {}
    config_dict["Evaluations"]["Corresponding_Days"] = corresponding_days
    config_dict["Evaluations"]["Baseline"] = thermalModel.baseline_error
    config_dict["Evaluations"]["Model"] = thermalModel.model_error
    config_dict["Evaluations"]["ActionOrder"] = thermalModel.scoreTypeList
    config_dict["Evaluations"]["Better Than Baseline"] = thermalModel.betterThanBaseline

    with open("./thermal_model_" + file_name, 'wb') as ymlfile:
        # TODO Note import pyaml here to get a pretty config file.
        try:
            import pyaml
            pyaml.dump(config_dict, ymlfile)
        except ImportError:
            yaml.dump(config_dict, ymlfile)
            
days = sorted(4 * list(range(1, 101, 5)))
# TODO change file name
# save_thermal_model_data(thermalModel, days, params, "Animal_shelter" + "_average_adding.yml")


In [4]:
# CONSISTANCY CHECKS

def apply_consistency_check_to_data(data, thermal_model=None, thermal_model_class=None):
    """
    :param data: Only used for fitting the thermal model."""
    assert not (thermal_model is None and thermal_model_class is None)
    
    # evaluate actions. On same temperatures, heating should increase, cooling decrease, and no action should be no different
    if thermal_model is None:
        thermalModel = thermal_model_class()
        thermalModel.fit(data, data["t_next"])
    else:
        thermalModel = thermal_model
    
    def prepare_consistency_test_data(thermal_model, start_temperature=50, end_temperature=100, increments=5):
        filter_columns = thermal_model._filter_columns
        data = []
        for temperature in range(start_temperature, end_temperature+increments, increments):
            # TODO potentially not hardcode dt
                for dt in range(5,20, 5):
                     for action in range(0, 3):
                        datapoint = {"dt": dt, "a1": int(0 < action <= 1), "a2": int(1 < action <= 2),
                             "t_out": temperature, "t_in": temperature}
                        for col in filter_columns:
                            if "zone_temperature_" in col:
                                datapoint[col] = temperature
                        data.append(datapoint)
        return pd.DataFrame(data)

    consistancy_test_data = prepare_consistency_test_data(thermalModel)
    consistancy_test_data["prediction"] = thermalModel.predict(consistancy_test_data, should_round=False)

    def consistency_check(df):
        """Consistency check for a df with 3 entries. The datapoints can only differ in the action to meaningful.
        :param df: pd.df columns as given by ControllerDataManger plus a column with the predctions"""
        t_in = df['t_in'].values[0]
        dt = df["dt"].values[0]
        heating_temperature = df[df['a1'] == 1]["prediction"].values
        cooling_temperature = df[df['a2'] == 1]["prediction"].values
        no_action_temperature = df[(df['a2'] != 1) & (df['a1'] != 1)]["prediction"].values
        consistency_flag = True
        
        # TODO only use this check when t_out and zone temperature are the same as t_in
        # Following checks with t_in are only possible when everything has the same temperature
        # check if predicted heating temperature is higher than current
        if heating_temperature <= t_in:
            consistency_flag = False
            print("Warning, heating_temperature is lower than t_in.")
        if cooling_temperature >= t_in:
            consistency_flag = False
            print("Warning, cooling_temperature is higher than t_in.")
        
        
        # check that heating is more than no action and cooling
        if heating_temperature <= no_action_temperature or heating_temperature <= cooling_temperature:
            consistency_flag = False
            print("Warning, heating_temperature is too low compared to other actions.")
        # check cooling is lower than heating and no action
        if cooling_temperature >= no_action_temperature or cooling_temperature >= heating_temperature:
            consistency_flag = False
            print("Warning, cooling_temperature is too high compared to other actions.")
        # check if no action is between cooling and heating
        if not cooling_temperature < no_action_temperature < heating_temperature:
            consistency_flag = False
            print("Warning, no action is not inbetween heating temperature and cooling temperature.")
        
        # want to know for what data it didn't work
        if not consistency_flag:
            print("Inconsistency for following data:")
            print(df)
            print("")
        return consistency_flag



    consistentcy_results = consistancy_test_data.groupby(["t_in", "dt"]).apply(lambda df: consistency_check(df))
    is_zone_consistent = all(consistentcy_results.values)
    if is_zone_consistent:
        print("The thermal model is consistent.")
    else:
        print("The thermal model is inconsistent.")


def check_consistency_building(thermal_data, thermal_model=None, thermal_model_class=None):
    assert not (thermal_model is None and thermal_model_class is None)
    for zone, data in thermal_data.items():
        print("--------- Consistency check for zone %s ---------" % zone)
        apply_consistency_check_to_data(data, thermal_model=thermal_model, thermal_model_class=thermal_model_class)



In [5]:
# TODO give each function a thermal model param so we can input the class/module.
def get_zone_scores(zone_data, test_on_training=True, thermal_model=None, thermal_model_class=None):
    assert not (thermal_model is None and thermal_model_class is None)
    if thermal_model is None:

        thermalModel = thermal_model_class()
        # ===== Use cross eval

    #         thermalModel = ThermalModel()
    #         print(cross_val_score(thermalModel, zone_data, zone_data['t_next']))

        # ===== end cross eval

        # ===== Use when not using cross eval function.
        if not test_on_training:
            PERCENT_TRAINING = 0.8
            PERCENT_TEST = 1  - PERCENT_TRAINING
        else:
            PERCENT_TRAINING = PERCENT_TEST = 1

        NUM_DATA = zone_data.shape[0]

        # NOTE: TAKING DATA AS GIVEN. NEED TO SHUFFLE OUTSIDE TO GET DIFFERENT RESULTS.
        training_data = zone_data.iloc[:int(NUM_DATA * PERCENT_TRAINING)]
        test_data = zone_data.iloc[-int(NUM_DATA * PERCENT_TEST):]

        thermalModel.fit(training_data, training_data['t_next'])
        
    else:
        thermalModel = thermal_model
        test_data = zone_data
    
    for score_action in range(-1, 3):
        thermalModel.score(test_data, test_data['t_next'], scoreType=score_action)

    # ==== end ====

    pprint.pprint("ScoreType List: ")
    pprint.pprint(thermalModel.scoreTypeList)
    print("")

    pprint.pprint("Better than baseline: ")
    pprint.pprint(thermalModel.betterThanBaseline)
    print("")
    
    
    
#     idx = pd.MultiIndex.from_product([['Model', 'Basline'],
#                                   thermalModel.scoreTypeList],
#                                  names=['Model Type', 'Action Eval'])
    
    df_model_error = pd.DataFrame(thermalModel.model_error, index=thermalModel.scoreTypeList)
    df_baseline_error = pd.DataFrame(thermalModel.baseline_error, index=thermalModel.scoreTypeList)

    
    df_model_baseline_error = pd.concat([df_model_error, df_baseline_error], axis=0, keys=["Model","Baseline"])
    print("Model and Baseline Error:")
    print(df_model_baseline_error)
    print("")
    
#     pprint.pprint("Model Error: ")
#     pprint.pprint(thermalModel.model_error)
#     print("")

#     pprint.pprint("Baseline Error: ")
#     pprint.pprint(thermalModel.baseline_error)
#     print("")


#         save_thermal_model_data(thermalModel, [45], thermalModel._params, "avenal-animal-shelter" + "_cross_eval_" + "_average_all.yml")
#         print(thermalModel.past_coeff)


def get_building_scores(thermal_data, test_on_training=False, thermal_model=None, thermal_model_class=None):
    for zone, zone_data in thermal_data.items():
        print("--------- Scoring Zone: %s ---------" % zone)
        get_zone_scores(zone_data, test_on_training=test_on_training, thermal_model_class=thermal_model_class, thermal_model=thermal_model)
        # Find best number of days to get for good thermalModel performance
        # TODO Right now only doing it for 50 days really


In [24]:
def zone_evaluate_action(zone_data):
    """Find the mean increase in temperature for each action for the given zone data"""
    def get_delta_mean(action_data):
        # get the mean change of temperature from now to next.    
        return np.mean((action_data["t_next"] - action_data["t_in"])/action_data["dt"])
    
    cooling_data = zone_data[zone_data["a2"] == 1]
    heating_data = zone_data[zone_data["a1"] == 1]
    no_action_data = zone_data[(zone_data["a1"] == 0) & (zone_data["a2"] == 0)]
    
    mean_cooling_delta = get_delta_mean(cooling_data)
    mean_heating_delta = get_delta_mean(heating_data)
    mean_no_action_delta = get_delta_mean(no_action_data)
    mean_all_action_delta = get_delta_mean(zone_data)
    
    print("For cooling there was an average %s degree change." % str(mean_cooling_delta))
    print("For heating there was an average %s degree change." % str(mean_heating_delta))
    print("For no action there was an average %s degree change." % str(mean_no_action_delta))
    print("For all actions there was an average %s degree change." % str(mean_all_action_delta))
    


# do data evaluation
def evaluate_action_impact(cfg=None, building=None, thermal_data=None):
    # Make sure we have either a config file to get the name or a building.
    for zone, data in thermal_data.items():
        print("----------- Get averages for zone: %s -----------" % zone)
        zone_evaluate_action(data)

In [7]:
# Look at a specific file
def get_data(building=None, cfg=None, days_back=50, evaluate_preprocess=False):
    assert cfg is not None or building is not None    
    if cfg is not None:
        building = cfg["Building"]
    else:
        try:
            with open("../Buildings/"+building+"/"+building+".yml", "r") as f:
                cfg = yaml.load(f)
        except:
            print("ERROR: No config file for building %s" % building)
            return

    print("----- Get data for Building: %s -----" % building)
    
    if evaluate_preprocess:
        path = "./Eval_data/"+building+"_eval"
    else:
        path = "./Eval_data/"+building
        
#     For raw data
#     c = get_client()
#     dataManager = ControllerDataManager(cfg, c)
#     return dataManager._get_inside_data(dataManager.now - datetime.timedelta(days=50), dataManager.now)
        
    # TODO ugly try/except   
    try:
        with open(path, "r") as f:
            import pickle
            thermal_data = pickle.load(f)
    except:
        c = get_client()
        dataManager = ControllerDataManager(cfg, c)
        thermal_data = dataManager.thermal_data(days_back=days_back, evaluate_preprocess=evaluate_preprocess)
        with open(path, "wb") as f:
            import pickle
            pickle.dump(thermal_data, f)
    return thermal_data

def concat_zone_data(thermal_data):
    """Concatinates all zone data into one big dataframe. Will sort by index. Get rid of all zone_temperature columns.
    :param thermal_data: {zone: pd.df}
    :return pd.df without zone_temperature columns"""
    concat_data =  pd.concat(thermal_data.values()).sort_index()
    filter_columns = ["zone_temperature" not in col for col in concat_data.columns]
    return concat_data[concat_data.columns[filter_columns]]

In [83]:
def apply_temperature_change(df):
    no_change_data = df[df["t_next"] == df["t_in"]]
    increase_data = df[df["t_next"] > df["t_in"]]
    decrease_data = df[df["t_next"] < df["t_in"]]
    to_return = {"total": df.shape[0],
                "total_no_change": no_change_data.shape[0],
                "percent_no_change": 100 * no_change_data.shape[0]/float(df.shape[0]),
                "total_increase": increase_data.shape[0],
                "percent_increase": 100 * increase_data.shape[0]/float(df.shape[0]),
                "total_decrease": decrease_data.shape[0],
                "percent_decrease": 100 * decrease_data.shape[0]/float(df.shape[0])}
    return pd.Series(to_return)

def no_temperature_change_interval(df):
    def no_temp_change_for_action(a_df):
        no_change = a_df[a_df["t_min"] == a_df["t_max"]]
        return pd.Series({"total": a_df.shape[0], "percent_no_change": 100 * no_change.shape[0]/float(a_df.shape[0]), 
                          "total_no_change": no_change.shape[0]})
    return df.groupby("action").apply(no_temp_change_for_action)



def evaluate_zone_data(data, is_building=False):
    """Does basic data evaluation.
    :param data: (pd.df) data for one zone. columns: (t_in, t_next, t_out, a1, a2, zone_temperature+zones* )"""
    # Filter for action data
    no_action_data = data[(data["a1"]==0) & (data["a2"]==0)]
    heating_data = data[data["a1"] == 1]
    cooling_data = data[data["a2"] == 1]
    
    # ====== Number and percentage of individual action data ========
    print("------------------")
    num_data = data.shape[0]
    num_no_action_data = no_action_data.shape[0]
    num_heating_data = heating_data.shape[0]
    num_cooling_data = cooling_data.shape[0]
    
    print("We have %f total data points." % num_data)
    
    print("We have %f no action data points, which is %f percent of total." % (num_no_action_data, 100 * float(num_no_action_data)/num_data))
    
    print("We have %f heating data points, which is %f percent of total." % (num_heating_data, 100 * float(num_heating_data)/num_data))

    print("We have %f cooling data points, which is %f percent of total." % (num_cooling_data, 100 * float(num_cooling_data)/num_data))
    # ========= end ==========
    
    print("------------------")
    
    # ========= Find temperature change by action ========
    no_action_change = apply_temperature_change(no_action_data)
    heating_change = apply_temperature_change(heating_data)
    cooling_change = apply_temperature_change(cooling_data)
    
    print("No action temperature changes:")
    print(no_action_change)
    print("")
    
    print("Heating temperature changes:")
    print(heating_change)
    print("")
    
    print("Cooling temperature changes:")
    print(cooling_change)
    print("")
    
    # ======== end =======
    
    # Group all data by dt and find no action change. 
    def group_dt_and_action(to_group_data):
        """Group the data by the interval lengths and whether temperature increased, dropped or stayed the same."""
        return to_group_data.groupby(by=["dt"]).apply(apply_temperature_change)
    
    no_action_dt_change = group_dt_and_action(no_action_data)
    heating_dt_change = group_dt_and_action(heating_data)
    cooling_dt_chage = group_dt_and_action(cooling_data)
    
    print("No Action, Action Change data:")
    print(no_action_dt_change)
    print("")
    
    print("Heating, Action Change data:")
    print(heating_dt_change)
    print("")
    
    print("Cooling, Action Change data:")
    print(cooling_dt_chage)
    print("")
    
    # ====== end =======
    
    if not is_building:
        # ====== Find actions which are not integers and stand by themselves
        print("===== Find non interger actions with no action pre/suceeding")
        for i in range(1,len(data.index)-1):
            curr = data["action"][i]
            prev = data["action"][i-1]
            next_a = data["action"][i+1]
            if curr not in [0,1,2] and prev == 0 and next_a == 0:
                print("========")
                print("Action is by itself with action %f for data:" % curr)
                print(data.iloc[i-2:i+2])

        # ====== end =======
    
    # ======= Find intervals where there was no change in temperature throughout the interval =======
    
    print()
    print(data.groupby("dt").apply(no_temperature_change_interval))
    
    

    

In [82]:
bldg = "avenal-veterans-hall"
zone_thermal_data = get_data(building=bldg, days_back=50, evaluate_preprocess=True)
building_thermal_data = concat_zone_data(zone_thermal_data)
# building_thermal_data = building_thermal_data[building_thermal_data["dt"]>2]

evaluate_zone_data(building_thermal_data, is_building= True)

# d = building_thermal_data[(building_thermal_data["a2"]==1) & (building_thermal_data["t_in"] == building_thermal_data["t_next"])]
# print(d[["t_min", "t_max", "dt", "action"]])

# avg_thermal_model = AverageThermalModel().fit(building_thermal_data, building_thermal_data["t_next"])
# avg_thermal_model.predict(building_thermal_data)
# get_building_scores(thermal_data=zone_thermal_data, thermal_model = avg_thermal_model)
# s_a_thermal_model = SuperAverageThermalModel()
# s_a_thermal_model.fit(thermal_data)
# s_a_thermal_model._params
# get_building_scores(thermal_data=thermal_data, thermal_model=s_a_thermal_model)

----- Get data for Building: avenal-veterans-hall -----
------------------
We have 37090.000000 total data points.
We have 31020.000000 no action data points, which is 83.634403 percent of total.
We have 716.000000 heating data points, which is 1.930439 percent of total.
We have 5354.000000 cooling data points, which is 14.435158 percent of total.
------------------
No action temperature changes:
percent_decrease        24.023211
percent_increase        44.081238
percent_no_change       31.895551
total                31020.000000
total_decrease        7452.000000
total_increase       13674.000000
total_no_change       9894.000000
dtype: float64

Heating temperature changes:
percent_decrease       0.418994
percent_increase      14.804469
percent_no_change     84.776536
total                716.000000
total_decrease         3.000000
total_increase       106.000000
total_no_change      607.000000
dtype: float64

Cooling temperature changes:
percent_decrease       74.075458
percent_increas

In [71]:
# data = concat_zone_data(zone_thermal_data)
# print(data[(data["t_max"] == data["t_min"]) & (data["action"] != 0)])
# print(data.groupby("action").count().iloc[:])


zone = "HVAC_Zone_AC-3"
# print(building_thermal_data.groupby("action").count())
data = zone_thermal_data[zone]
f_data = data[(data["action"]>0.331)&(data["action"]<0.334)]
idx = f_data.index[20]
print("============")
print(zone)
print(idx)
print()
# print(data.loc[idx - datetime.timedelta(hours=1):idx + datetime.timedelta(hours=1)])

for i in range(1,len(data.index)-1):
    curr = data["action"][i]
    prev = data["action"][i-1]
    next_a = data["action"][i+1]
    if curr not in [0,1,2] and prev == 0 and next_a == 0:
        print("========")
        print(i)
        print(curr)
print(data.iloc[5659:5663])

# print(building_thermal_data[(building_thermal_data["action"]>1.636)&(building_thermal_data["action"]<1.637)])
# print(data[(0 < data["action"]) & (data['action'] <= 1)])


# inside_data = get_data(building=bldg)

HVAC_Zone_AC-3
2018-05-09 13:15:14-07:00
()
1317
1.6666666666666667
1340
1.6666666666666667
1349
1.6666666666666667
1673
1.6666666666666667
1675
1.6666666666666667
4380
1.5
5661
0.2222222222222222
                             action  dt       t_in      t_max     t_mean  \
time                                                                       
2018-06-08 19:17:14-07:00  1.176471   1  68.023529  68.023529  68.023529   
2018-06-08 19:18:14-07:00  0.000000  14  68.164706  69.400000  68.700359   
2018-06-08 19:32:14-07:00  0.222222   1  69.400000  69.400000  69.400000   
2018-06-08 19:33:14-07:00  0.000000   2  69.400000  69.400000  69.400000   

                               t_min     t_next      t_out     t_std  \
time                                                                   
2018-06-08 19:17:14-07:00  68.023529  68.023529  80.000636  0.000000   
2018-06-08 19:18:14-07:00  68.100000  69.400000  80.000636  0.388499   
2018-06-08 19:32:14-07:00  69.400000  69.400000  80.000636

In [39]:
c_inside_data = concat_zone_data(inside_data)
c_inside_data.groupby("a").count()

,t_in
a,
0.000000,355767
0.055556,1
0.083333,11
0.090909,4
0.111111,41
0.117647,35
0.125000,10
0.166667,103
0.181818,85


In [99]:

building = "avenal-animal-shelter"
# zone = "HVAC_Zone_AC-2"
thermal_data = get_data(building=building)
zone = thermal_data.keys()[0]

zone_data = thermal_data[zone]

filtered_zone_data = zone_data[zone_data["dt"] == 15]

heating_thermal_data = {}
# for zone, zone_data in thermal_data.items():
#     # change following to action we want
#     heating_thermal_data[zone] = zone_data
# #     heating_thermal_data[zone] = zone_data[(zone_data["a1"]==0) & (zone_data["a2"]==0)]
# #     heating_thermal_data[zone] = zone_data[zone_data["a1"] == 1]
    
print("--- not filtered ---")
evaluate_zone_data(zone_data)
print("--- filtered ---")
evaluate_zone_data(filtered_zone_data)

f_n = filtered_zone_data[(filtered_zone_data["a1"]==0) & (filtered_zone_data["a2"]==0)]
n = zone_data[(zone_data["a1"]==0) & (zone_data["a2"]==0)]
print("+++++++++++++++")
print("not filtered: ", np.sqrt(np.mean(np.square(n["t_next"] - n["t_in"]))))
print("filtered: ", np.sqrt(np.mean(np.square(f_n["t_next"] - f_n["t_in"]))))
print("+++++++++++++++")






print("++++++++ Standard Model ++++++++")
get_building_scores(ThermalModel, thermal_data={zone: zone_data}, test_on_training=True)
print("---- Filtered Data ----")
get_building_scores(ThermalModel, thermal_data={zone: filtered_zone_data}, test_on_training=True)
# check_consistency_for_cfg(thermal_data=heating_thermal_data)

print("++++++++ Average Model ++++++++")
get_building_scores(AverageThermalModel, thermal_data={zone: zone_data}, test_on_training=True)
print("---- Filtered Data ----")
get_building_scores(AverageThermalModel, thermal_data={zone: filtered_zone_data}, test_on_training=True)
# check_consistency_for_cfg(thermal_data=heating_thermal_data)



print("---- zone action %s ------" % zone)
zone_evaluate_action(thermal_data[zone])

# [0.4345687  0.25142704 0.26405015]

----- Get data for Building: avenal-animal-shelter -----
--- not filtered ---
------------------
We have 5320.000000 total data points.
We have 4657.000000 no action data points, which is 87.537594 percent of total.
We have 195.000000 heating data points, which is 3.665414 percent of total.
We have 468.000000 cooling data points, which is 8.796992 percent of total.
------------------
No Action No Change data:
    percent_no_change   total  total_no_change
dt                                            
1          100.000000   912.0            912.0
2           10.256410    39.0              4.0
3           21.052632    19.0              4.0
4           40.000000    10.0              4.0
5           25.000000     4.0              1.0
6           11.111111     9.0              1.0
7            6.666667    15.0              1.0
8            9.523810    21.0              2.0
9           11.764706    17.0              2.0
10          15.384615    13.0              2.0
11           0.000000  

In [53]:
# Going through all config files

import os
# Go to folder where all config files are located
config_folder_location = "../Buildings/"
all_dir = os.walk(config_folder_location).next()
for d in all_dir[1]:
    print("============================================================================")
    print("============================================================================")
    end_dir = config_folder_location + d + "/"

    files = os.walk(end_dir).next()[2]
    print("in dir: ", end_dir)
    for f in files:
        if ".yml" not in f:
            print("%s is not a yaml file. Continue to next file." % f)
            continue
        print("Getting file %s" % f)
        # Loads the configs
        with open("./" + end_dir + "/" + f, 'r') as o:
            config = yaml.load(o)
        
        # Do whatever check on config
        building = config["Building"]
        
        zone_thermal_data = get_data(building=building)
        
        
        
        
        filtered_thermal_data = {}
        for zone, data in zone_thermal_data.items():
            filtered_thermal_data[zone] = data[data["dt"] != 1]
            
        buidling_thermal_data = concat_zone_data(filtered_thermal_data)


        
#         print("================== Scores for Building: %s ==================" % building)      
#         get_scores_for_days(thermal_data=thermal_data)

#         print("================== Evaluate data for Buidling: %s ==================" % building)
#         for zone, zone_data in thermal_data.items():
#             print("---------- Evaluating data for zone %s ----------" % zone)
#             evaluate_zone_data(zone_data)
            
        th = AverageThermalModel().fit(buidling_thermal_data, buidling_thermal_data["t_next"])
        
        print("================== Consistency check for Building: %s ==================" % building)   
        check_consistency_building(thermal_data=filtered_thermal_data, thermal_model=th)


        
#         print("================== Get averages for Building: %s ==================" % building)        
#         evaluate_action_impact(thermal_data=thermal_data)
        


        
        


('in dir: ', '../Buildings/ciee/')
.DS_Store is not a yaml file. Continue to next file.
Getting file ciee.yml
----- Get data for Building: ciee -----
================== Consistency check for Building: ciee ==================
--------- Consistency check for zone HVAC_Zone_Eastzone ---------
The thermal model is consistent.
--------- Consistency check for zone HVAC_Zone_Southzone ---------
The thermal model is consistent.
--------- Consistency check for zone HVAC_Zone_Centralzone ---------
The thermal model is consistent.
--------- Consistency check for zone HVAC_Zone_Northzone ---------
The thermal model is consistent.
('in dir: ', '../Buildings/avenal-recreation-center/')
Getting file avenal-recreation-center.yml
----- Get data for Building: avenal-recreation-center -----
================== Consistency check for Building: avenal-recreation-center ==================
--------- Consistency check for zone HVAC_Zone_Large_Room ---------
The thermal model is consistent.
--------- Consistency

In [13]:
a = np.array([(1.0,), (1.0,), (1.0,), (1.0000000149011612,), (-2.911192977506083e-05,), (-2.9111929341259272e-05,), (-2.9234661707105666e-05,), (-2.911192977506083e-05,), (-2.911192977506083e-05,), (-2.911192977506083e-05,), (-2.911192977506083e-05,)])
import matplotlib.pyplot as plt
plt.plot(a)
plt.show()

In [ ]:
thermal_model_zones = {zone: ThermalModel().fit(X=data, y=data["t_next"]) for zone, data in thermal_data.items()}

In [ ]:
def getBestHorizon(data):
    """Finds the best stretch of data, i.e. with most actions which are either heating or cooling. Gives
    preference to cooling.
    returns: best data slice"""
    horizon = datetime.timedelta(hours=3)
    start = datetime.datetime(year=2018, month=3, day=9, hour=8, minute=15)

    good_dates = []
    delta_minutes = datetime.timedelta(minutes=15)
    for row in data.itertuples():
        start = row[0]
        if sum(data.loc[start:start+horizon]["action"]) > 0:
            good_dates.append((start, sum(data.loc[start:start+horizon]["action"])))
    good_dates.sort(key=lambda x: x[1])
    new_start = good_dates[-1][0]
    return data.loc[new_start:new_start+horizon]

In [ ]:
trivial_predictions = {zone: np.ones(data.shape[0]) * data["t_in"][0] for zone, data in thermal_data.items()} # we predict the first temperature for the whole horizon


In [ ]:
def constantZonePredictions(data, thermal_model):
    zone_temp_filter = data.columns[["zone_temperature_" in col for col in data.columns]]
    constant_zone_temperatures = data[zone_temp_filter].iloc[0] # get the first row for constant zone temperatures. 

    constant_zone_predictions = [data.iloc[0]["t_in"]]
    for row in data.iterrows():
        # constant zone temperature predictions
        row_data = row[1] # work with the row data
        row_data["t_in"] = constant_zone_predictions[-1] # assign last prediction
        row_data[constant_zone_temperatures.index] = constant_zone_temperatures.values
        constant_zone_predictions.append(thermal_model.predict(row_data))
    return constant_zone_predictions

def perfectZonePredictions(data, thermal_model):
    perfect_zone_predictions = [data.iloc[0]["t_in"]]
    for row in data.iterrows():
        row_data = row[1] # work with the row data
        # assume we know the zone temperatures perfectly
        row_data["t_in"] = perfect_zone_predictions[-1] # assign last prediction
        perfect_zone_predictions.append(thermal_model.predict(row_data))
    return perfect_zone_predictions



In [ ]:
southzone_best_data = getBestHorizon(thermal_data["HVAC_Zone_Southzone"]) # use some interval
start = southzone_best_data.index[0]
end = southzone_best_data.index[-1]
bestData = {zone: data[start:end] for zone, data in thermal_data.items()}
rename_col = {zone: "zone_temperature_" + zone for zone in thermal_data.keys()}
for zone in thermal_data.keys():
    for zone1 in thermal_data.keys():
        ti = "zone_temperature_" + zone1
        if ti not in southzone_best_data.columns and zone != zone1:
            bestData[zone] = southzone_best_data.rename({"t_in" : ti, "zone_temperature_"+ zone: "t_in"}, axis='columns')

In [ ]:
print(bestData)

In [ ]:
# bestData = {zone: southzone_best_data for zone, data in thermal_data.items()}

# iteration evaluation
# start by assuming that we only know current zone temperatures. Find predictions for each zone that way
last_prediction_data = {zone: constantZonePredictions(data, thermal_model_zones[zone]) for zone, data in bestData.items()}
print(last_prediction_data)
for _ in range(1):
    temp_prediction_data = {}
    for zone, data in bestData.items():
        for prediction_zone, prediction in last_prediction_data.items():
            if prediction_zone != zone:
                data.loc[:,"zone_temperature_" + prediction_zone] = prediction[:-1] # TODO NOTE we predict one more than the data we have. 
        print(data)
        temp_prediction_data[zone] = perfectZonePredictions(data, thermal_model_zones[zone])
    last_prediction_data = temp_prediction_data
        
print(last_prediction_data)

In [ ]:
d = getBestHorizon(thermal_data["HVAC_Zone_Southzone"])
print(constantZonePredictions(d, thermal_model_zones["HVAC_Zone_Southzone"]))

print(trivial_prediction)
print(constant_zone_predictions)
print(perfect_zone_predictions)

In [ ]:
trivial_error = []
constant_error = []
perfect_error = []

dt = data["dt"]
t_in = data["t_in"]
for i in range(1,len(t_in)):
    trivial_error.append(thermal_model._normalizedRMSE_STD(trivial_prediction[:i], t_in[:i], dt[:i]))
    constant_error.append(thermal_model._normalizedRMSE_STD(constant_zone_predictions[:i], t_in[:i], dt[:i]))
    perfect_error.append(thermal_model._normalizedRMSE_STD(perfect_zone_predictions[:i], t_in[:i], dt[:i]))

In [ ]:
diff = constant_zone_predictions[:-1] - t_in[:]
print(np.square(diff))

plt.plot(trivial_error)
plt.show()

print(constant_error)
plt.plot(constant_error)
plt.show()

plt.plot(perfect_error)
plt.show()

In [ ]:
np.array(constant_error)[:, 1]

$$T_{in} + dt * ( a_1*c_1 + a_2*c_2 + (T_{out} - T_{in})*c_3 + c_4 + \sum_{i = 0}^N (T_{zone_i} - T_{in}) * c_{5+i})$$

In [130]:
a = np.array([[1,2,3], [5432,31, 2]])

In [133]:
np.mean(a, axis=0)

array([2.7165e+03, 1.6500e+01, 2.5000e+00])